In [3]:
import pandas as pd
import numpy as np
import re
import editdistance as ed
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from spacy.lang.en import English
import nltk
nltk.download('wordnet')
from nltk import word_tokenize
nltk.download('punkt')
from nltk.corpus import wordnet 
import spacy
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer
from snorkel.labeling import PandasLFApplier, LFApplier, LFAnalysis, labeling_function
from snorkel.analysis import get_label_buckets

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\I517193\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\I517193\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Schema
* The data consist of the queries users searched for, by voice, converted to text
* All queries are labeled with their tags (each unigram is labeled with the entity name)
* The entities covered are as follows:
  * Restuarant_name
  * Rating
  * Amenity
  * Location
  * Price
  * Hours
  * Dish
  * Cuisine

**Preprocessing the file**

* Separating each candidate noun phrase from the query

In [4]:
def ret_df_file(file):
    with open(file, "r") as f:
        lines = f.readlines()
        size = len(lines) 
        idx_list = [idx + 1 for idx, val in
                enumerate(lines) if val == '\n'] 
        res = [lines[i: j-1] for i, j in
            zip([0] + idx_list, idx_list + 
            ([size] if idx_list[-1] != size else []))] 
        queries = ['' for i in range(len(res))]
        tags = []
        unigrams = []
        words = []
        poss = []
        for i, ele in enumerate(res):
            flag = False
            tempWords = []
            temT = []
            temW = []
            temPos = []
            for j, actual_str in enumerate(ele):
                res[i][j] = actual_str.split('\t')
                res[i][j][-1] = res[i][j][-1].replace('\n', '')
                if res[i][j][0][:2]=="B-" or res[i][j][0][:2]=="I-":
                    if flag:
                        if tempTag == res[i][j][0][2:]:
                            tempWords.append(res[i][j][-1])
                        else:
                            temT.append(tempTag)
                            temW.append(' '.join(tempWords))
                            pos[-1] = j-1
                            temPos.append(pos)
                            tempWords = []
                            pos = [j,j]
                            tempWords.append(res[i][j][-1])
                            tempTag = res[i][j][0][2:]
                        # tempTag.append
                    else:
                        tempWords = []
                        tempWords.append(res[i][j][-1])
                        pos = [j,j]
                        tempTag = res[i][j][0][2:]
                        flag = True
                else:
                    if flag:
                        temT.append(tempTag)
                        temW.append(' '.join(tempWords))
                        pos[-1] = j-1
                        temPos.append(pos)
                        flag = False
                if j!=0:
                    queries[i] = queries[i] + ' ' + res[i][j][-1]
                else:
                    queries[i] = res[i][j][-1]
            if flag:
                temT.append(tempTag)
                temW.append(' '.join(tempWords))
                pos[-1] = j
                temPos.append(pos)
                flag = False
            tags.append(temT)
            words.append(temW)
            poss.append(temPos)
    df_queries = []
    df_pos_start = []
    df_pos_end = []
    n_grams = []
    df_tags = []
    for i, ele in enumerate(res):
        for j in range(len(tags[i])):
            df_queries.append(queries[i])
            df_tags.append(tags[i][j])
            df_pos_start.append(poss[i][j][0])
            df_pos_end.append(poss[i][j][1])
            n_grams.append(words[i][j])
    return pd.DataFrame({"n_grams": n_grams, "Tag": df_tags, "Query": df_queries, "Pos_start": df_pos_start, "Pos_end":df_pos_end})

**Getting the train and test data**

In [5]:
df_train = ret_df_file("../Datasets/MITRestuarant/restauranttrain.bio")
df_test = ret_df_file("../Datasets/MITRestuarant/restauranttest.bio")

In [6]:
print(df_train.shape)
print(df_test.shape)

(15363, 5)
(3151, 5)


In [7]:
df_train.head(5)

,n_grams,Tag,Query,Pos_start,Pos_end
0,2 start,Rating,2 start restaurants with inside dining,0,1
1,inside dining,Amenity,2 start restaurants with inside dining,4,5
2,5 star,Rating,5 star resturants in my town,0,1
3,in my town,Location,5 star resturants in my town,3,5
4,hong kong,Restaurant_Name,98 hong kong restaurant reasonable prices,1,2


In [8]:
df_test.head(5)

,n_grams,Tag,Query,Pos_start,Pos_end
0,four star,Rating,a four star restaurant with a bar,1,2
1,with a,Location,a four star restaurant with a bar,4,5
2,bar,Amenity,a four star restaurant with a bar,6,6
3,asian,Cuisine,any asian cuisine around,1,1
4,around,Location,any asian cuisine around,3,3


**Now I extract candidate noun phrases from the queries using nlp module's noun chunks**

Also, to label the noun phrases, I am using the following assumptions:
* The labeled noun phrase and the noun phrase extracted using nlp should have an intersection of more than one word
* The absolute value of the difference in the number of words in the labelled and the extracted noun phrase should not be more than 2 words

If both the above criteria are satisfied, then I label the noun phrase given by the nlp module using the label from the noun phrase already labelled.

*Note: I need to do this as using the same noun phrase manually labelled would not be available at test time, and we should train the model in a similar way*

In [10]:
def obtain_noun_chunks_limited(df_train):
    list_of_queries = df_train.Query.unique()
    queries_train = []
    tags_train = []
    phrases_train = []
    pos_S = []
    pos_E = []
    nlp = spacy.load("en_core_web_sm")
    for query in list_of_queries:
        doc = nlp(query)
        positions_start = list(df_train.Pos_start[df_train.Query==query])
        positions_end = list(df_train.Pos_end[df_train.Query==query])
        for noun in doc.noun_chunks:
            flag = False
            for i in range(len(positions_start)):
                # checking intersection>0 and extra words in noun_chunk not more than 2 and difference 
                # between the phrases of not more than 2 words
                if len(set(list(range(noun.start, noun.end))) & \
                       set(list(range(positions_start[i], positions_end[i]+1))))>0 \
                and (len(set(list(range(noun.start, noun.end))).difference(set(list(range(positions_start[i], \
                                                                                            positions_end[i]+1)))))<=2 \
                    and len(set(list(range(positions_start[i], positions_end[i]+1))).difference(set(list(range(noun.start, noun.end)))))<=2):
                    queries_train.append(query)
                    tags_train.append(list(df_train.Tag[np.logical_and(df_train.Query==query, \
                                                                  df_train.Pos_start==positions_start[i])])[0])
                    phrases_train.append(noun.text)
                    pos_S.append(noun.start)
                    pos_E.append(noun.end-1)
                    flag = True
                    break
    return pd.DataFrame({"n_grams": phrases_train, "Tag": tags_train, \
                                "Query": queries_train, "Pos_start": pos_S, "Pos_end":pos_E})

In [11]:
df_train_limited_chunks = obtain_noun_chunks_limited(df_train)

In [12]:
df_valid_limited_chunks = obtain_noun_chunks_limited(df_test)

In [13]:
df_valid_limited_chunks.head()

,n_grams,Tag,Query,Pos_start,Pos_end
0,a four star restaurant,Rating,a four star restaurant with a bar,0,3
1,a bar,Location,a four star restaurant with a bar,5,6
2,any asian cuisine,Cuisine,any asian cuisine around,0,2
3,any bbq places,Cuisine,any bbq places open before 5 nearby,0,2
4,any dancing establishments,Location,any dancing establishments with reasonable pri...,0,2


**There are the following types of entities available in the data**

In [14]:
df_train_limited_chunks.Tag.unique()

array(['Rating', 'Amenity', 'Location', 'Hours', 'Dish', 'Cuisine',
       'Price', 'Restaurant_Name'], dtype=object)

**Defining the Labels**

Now we assign numeric labels to the entities and then change them in the dataset

In [15]:
RATING = 0
AMENITY = 1
LOCATION = 2
RESTUARANT_NAME = 3
HOURS = 4
DISH = 5
CUISINE = 6
PRICE = 7
ABSTAIN  = -1

In [16]:
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Rating'] = RATING
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Amenity'] = AMENITY
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Location'] = LOCATION
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Restaurant_Name'] = RESTUARANT_NAME
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Hours'] = HOURS
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Dish'] = DISH
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Cuisine'] = CUISINE
df_train_limited_chunks.Tag[df_train_limited_chunks.Tag=='Price'] = PRICE

## Writing Labelling functions

**Labelling functions for Rating**

In [17]:
@labeling_function()
def lf_rating_star(x):
    # Returns a label of rating if pattern of digit star's found in the phrase
    n_grams = x.n_grams.lower()
    if re.findall("[\d]+[\W]?star", n_grams):
        return RATING
    else:
        return ABSTAIN

In [18]:
@labeling_function()
def lf_rating_adj(x):
    # Returns a label of rating if any adjective is within one edit distance or the base form of adjective is found in the phrase
    adjectives = ['fresh', 'recommended', 'fair', 'recommendation', 'rated'
       'terrific', 'well', 'awesome', 'awards', 'authentic', 'pleasing', 'favorite',
        'lovely', 'really', 'decent', 'busy', 'superior','average'
       'famous', 'poor', 'decent', 'simple', 'real', 'popular', 'wonderful',
       'casual', 'perfect', 'massive', 'nearby', 'nice',
       'delicious', 'winning', 'favourite',  'negative', 'award', 'positive', 'fancy',
       'outstanding', 'good', 'amazing', 'recommend',
       'strong', 'value','incredible',
       'fantastic', 'classy', 'top', 'former', 'rated','reviews','horrible','terrible', 
       'local', 'excellent', 'place',
       'tasty', 'rate', 'high', 'great']
    lemmatiser = WordNetLemmatizer()
    for word in x.n_grams.lower().split():
        for adjective in adjectives:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return RATING
    return ABSTAIN

In [19]:
@labeling_function()
def lf_rating_adj_syn(x):
    # Returns a label of rating if any word in the phrase is a synonym or an antonym within one edit distance of the mentioned words
    adjectives = ['fresh', 'recommended', 'fair', 'recommendation', 'rated'
       'terrific', 'well', 'awesome', 'awards', 'authentic', 'pleasing', 'favorite',
        'lovely', 'really', 'decent', 'busy', 'superior',
       'famous', 'poor', 'decent', 'simple', 'real', 'popular', 'wonderful',
       'casual', 'perfect', 'massive', 'nearby', 'nice',
       'delicious', 'winning', 'favourite',  'negative', 'award', 'positive', 'fancy',
       'outstanding', 'good', 'amazing', 'recommend',
       'strong', 'value','incredible',
       'fantastic', 'classy', 'top', 'former', 'rated','reviews','horrible','terrible', 
       'local', 'excellent', 'place',
       'tasty', 'rate', 'high', 'great']
    synonyms = [] 
    antonyms = [] 
    for adj in adjectives:
        for syn in wordnet.synsets(adj): 
            for l in syn.lemmas(): 
                synonyms.append(l.name()) 
                if l.antonyms(): 
                    antonyms.append(l.antonyms()[0].name()) 
    
    lemmatiser = WordNetLemmatizer()
    for word in x.n_grams.lower().split():
        for adjective in synonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return RATING
        for adjective in antonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return RATING
    return ABSTAIN

**Labelling functions for Amenity**

In [20]:
@labeling_function()
def lf_amenity_lem(x):
    # Returns a label of amenity if any word is within one edit distance of the mentioned adjectives or it's lemmatized form
    amenities = ['formal', 'outdoor', 'friendly', 'parking', 'special'
       'quiet', 'smoking', 'atmosphere', 'anniversary', 'birthday', 'tourist', 'reservation']
    lemmatiser = WordNetLemmatizer()
    for word in x.n_grams.lower().split():
        for adjective in amenities:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return AMENITY
    return ABSTAIN

In [21]:
@labeling_function()
def lf_amenity_syn(x):
    # Returns a label of amenity if any word in the phrase is a synonym or an antonym of the mentioned words within one edit distance
    amenities = ['formal', 'outdoor', 'friendly', 'parking', 'special'
       'quiet', 'smoking', 'atmosphere', 'anniversary', 'birthday', 'tourist', 'reservation']

    synonyms = [] 
    antonyms = [] 
    for adj in amenities:
        for syn in wordnet.synsets(adj): 
            for l in syn.lemmas(): 
                synonyms.append(l.name()) 
                if l.antonyms(): 
                    antonyms.append(l.antonyms()[0].name()) 
    
    lemmatiser = WordNetLemmatizer()
    for word in x.n_grams.lower().split():
        for adjective in synonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return AMENITY
        for adjective in antonyms:
            if ed.eval(adjective, word)<=1 or lemmatiser.lemmatize(word, pos = "a")==adjective:
                return AMENITY
    return ABSTAIN

**Labeling function for Location**

In [22]:
@labeling_function()
def lf_location_prep_before(x):
    # Returns a label of location if the word before the phrase is a preposition mentioned
    if (x.Pos_start>=1 and x.Query.split()[x.Pos_start - 1].lower() in ['in', 'near', 'above', 'over', 'by', 'along', 'around']) or \
    (x.Pos_start>1 and x.Query.split()[x.Pos_start - 2].lower() in  ['in', 'near', 'above', 'over', 'by', 'along', 'on']):
        return LOCATION
    else:
        return ABSTAIN

In [23]:
@labeling_function()
def lf_location_prep_after(x):
    # Returns a label of location if the word after the phrase is the word "where"
    if x.Pos_end + 1 <= (len(x.Query.strip().split()) - 1) \
    and x.Query.split()[x.Pos_end + 1].lower() in ['where']:
        return LOCATION
    else:
        return ABSTAIN

**Labeling function for Hours**

In [24]:
@labeling_function()
def lf_hours_keywords(x):
    # Returns a label of hours if the the phrase contains any of the words mentioned below
    keywords = ['hour', 'hours', 'am', 'a.m.', 'pm', 'p.m.', 'early', 'late']
    for word in x.n_grams.lower().split():
        if word in keywords:
            return HOURS
    return ABSTAIN

**Labeling function for Price**

In [25]:
@labeling_function()
def lf_price_keywords(x):
    # Returns a label of price if the the phrase contains any of the words mentioned below
    keywords = ['cheap', 'expensive', 'reasonable', 'bucks', 'dollars', 'price', 'prices', 'affordable', \
                'midpriced', 'high', 'least']
    for word in x.n_grams.lower().split():
        if word in keywords:
            return PRICE
    return ABSTAIN

**Labeling function for cuisine**

In [27]:
@labeling_function()
def lf_cuisine_keywords(x):
    # Returns a label of cuisine if the the phrase contains any of the words mentioned below
    cuisine_words_list = ['Ainu', 'Albanian', 'Argentine', 'Andhra', 'Anglo-Indian', 'Arab', 'Armenian', 'Assyrian', 'Awadhi', \
                'Azerbaijani', 'Balochi', 'Belarusian', 'Bangladeshi', 'Bengali', 'Berber', 'Brazilian', 'Buddhist', \
                'Bulgarian', 'Cajun', 'Cantonese', 'Caribbean', 'Chechen', 'Chinese', 'Chinese Islamic', 'Circassian', \
                'Crimean Tatar', 'Cypriot', 'Danish', 'English', 'Estonian', 'French', 'Filipino', 'Georgian', 'German', \
                'Goan', 'Goan Catholic', 'Greek', 'Gujarati', 'Hyderabad', 'Hong Kong Western', 'Indian', 'Indian Chinese', \
                'Indian Singaporean', 'Indonesian', 'Inuit', 'Irish', 'Italian American', 'Italian', 'Jamaican', 'Japanese', \
                'Jewish', 'Karnataka', 'Kazakh', 'Keralite', 'Korean', 'Kurdish', 'Laotian', 'Lebanese', 'Latvian', \
                'Lithuanian', 'Louisiana Creole', 'Maharashtrian', 'Mangalorean', 'Malay', 'Malaysian Chinese', \
                'Malaysian Indian', 'Mediterranean', 'Mexican', 'Mordovian', 'Mughal', 'Native American', 'Nepalese', \
                'New Mexican', 'Odia', 'Parsi', 'Pashtun', 'Polish', 'Pennsylvania Dutch', 'Pakistani', 'Peranakan', \
                'Persian', 'Peruvian', 'Portuguese', 'Punjabi', 'Rajasthani', 'Romanian', 'Russian', 'Sami', 'Serbian', \
                'Sindhi', 'Slovak', 'Slovenian', 'Somali', 'South Indian', 'Soviet', 'Spanish', 'Sri Lankan', 'Taiwanese', \
                'Tatar', 'Thai', 'Turkish', 'Tamil', 'Udupi', 'Ukrainian', 'Vietnamese', 'Yamal', 'Zambian', 'Zanzibari', \
               'cuisine', 'cuisines']
    new_cuisine_list = []
    for word in cuisine_words_list:
        new_cuisine_list.append(word.lower())
    for word in x.n_grams.lower().split():
        if word in new_cuisine_list:
            return CUISINE
    return ABSTAIN

In [28]:
df_rating_amenity_train = df_train_limited_chunks[df_train_limited_chunks['Tag'].isin([0,1,2, HOURS, PRICE, CUISINE])]

In [29]:
df_train_lf_rating_amenity, df_test_lf_rating_amenity, y_lf_train, y_lf_test = train_test_split(df_rating_amenity_train.drop('Tag', axis = 1),\
                                                                                df_rating_amenity_train['Tag'], test_size = 0.3)

In [30]:
from snorkel.labeling import PandasLFApplier

lfs = [lf_rating_star, lf_rating_adj, lf_rating_adj_syn, lf_amenity_lem, lf_amenity_syn, \
       lf_location_prep_before, lf_location_prep_after, lf_hours_keywords, lf_price_keywords, lf_cuisine_keywords]

In [31]:
df_test_lf_rating_amenity.head()

,n_grams,Query,Pos_start,Pos_end
2236,expensive restraunt,expensive restraunt,0,1
4535,an affordable place,i would like to try nepalese cuisine today is ...,10,12
3676,alcohol,i need a family restaurant that serves alcohol,7,7
7278,a good restaurant,what is a good restaurant for upscale french c...,2,4
2376,two hours,find a place to eat in two hours,6,7


In [32]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train_lf_rating_amenity)
L_dev = applier.apply(df=df_test_lf_rating_amenity)

100%|██████████████████████████████████████████████████████████████████████████████| 2139/2139 [01:04<00:00, 33.08it/s]


In [33]:
coverage_stars, coverage_adjectives, coverage_synonyms, coverage_amenity_lem, coverage_amenity_syn, coverage_loc_before, \
coverage_loc_after, coverage_hours, coverage_price, coverage_cuisine = (L_train != ABSTAIN).mean(axis=0)
print(f"Labelling function stars coverage: {coverage_stars * 100:.1f}%")
print(f"Labelling function adjectives coverage: {coverage_adjectives * 100:.1f}%")
print(f"Labelling function synonyms of adjectives coverage: {coverage_synonyms * 100:.1f}%")
print(f"Labelling function lemmas of amenity coverage: {coverage_amenity_lem * 100:.1f}%")
print(f"Labelling function synonyms of amenity coverage: {coverage_amenity_syn * 100:.1f}%")
print(f"Labelling function synonyms of location_before_prep coverage: {coverage_loc_before * 100:.1f}%")
print(f"Labelling function synonyms of location_after_prep coverage: {coverage_loc_after * 100:.1f}%")
print(f"Labelling function synonyms of hours coverage: {coverage_hours * 100:.1f}%")
print(f"Labelling function synonyms of price coverage: {coverage_price * 100:.1f}%")
print(f"Labelling function synonyms of cuisine coverage: {coverage_cuisine * 100:.1f}%")

Labelling function stars coverage: 0.6%
Labelling function adjectives coverage: 21.5%
Labelling function synonyms of adjectives coverage: 38.8%
Labelling function lemmas of amenity coverage: 6.4%
Labelling function synonyms of amenity coverage: 8.1%
Labelling function synonyms of location_before_prep coverage: 15.1%
Labelling function synonyms of location_after_prep coverage: 0.8%
Labelling function synonyms of hours coverage: 2.3%
Labelling function synonyms of price coverage: 4.6%
Labelling function synonyms of cuisine coverage: 10.9%


In [34]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_rating_star,0,[0],0.006414,0.006414,0.001403
lf_rating_adj,1,[0],0.214672,0.214672,0.067148
lf_rating_adj_syn,2,[0],0.388054,0.260373,0.108238
lf_amenity_lem,3,[1],0.064342,0.064342,0.007416
lf_amenity_syn,4,[1],0.080577,0.077370,0.020445
lf_location_prep_before,5,[2],0.150732,0.022049,0.021247
lf_location_prep_after,6,[2],0.008018,0.005813,0.005011
lf_hours_keywords,7,[4],0.023452,0.008218,0.008218
lf_price_keywords,8,[7],0.045701,0.031068,0.031068
lf_cuisine_keywords,9,[6],0.109040,0.042093,0.042093


In [35]:
y_test_int = y_lf_test.astype('int32')

In [36]:
LFAnalysis(L=L_dev, lfs=lfs)\
.lf_summary(y_test_int)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_rating_star,0,[0],0.005610,0.005610,0.000468,12,0,1.000000
lf_rating_adj,1,[0],0.211781,0.211781,0.056568,145,308,0.320088
lf_rating_adj_syn,2,[0],0.402992,0.259935,0.100514,198,664,0.229698
lf_amenity_lem,3,[1],0.056101,0.056101,0.007013,120,0,1.000000
lf_amenity_syn,4,[1],0.072931,0.069191,0.020103,128,28,0.820513
lf_location_prep_before,5,[2],0.163160,0.029921,0.028986,329,20,0.942693
lf_location_prep_after,6,[2],0.005610,0.004675,0.003740,3,9,0.250000
lf_hours_keywords,7,[4],0.031323,0.006078,0.006078,60,7,0.895522
lf_price_keywords,8,[7],0.034596,0.021973,0.021973,62,12,0.837838
lf_cuisine_keywords,9,[6],0.100514,0.039271,0.039271,205,10,0.953488
